In [18]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine, text
import warnings

warnings.filterwarnings("ignore")

In [19]:
user_name='giovani_goltara'
password='ZXBq7H2Nr189bs'

In [20]:
# Conection
host = 'localhost'
port = '5433'
database = 'layereddb'
schema='berlin_source_data'

# Connection to db using SQLAlchemy
engine = create_engine(f'postgresql+psycopg2://{user_name}:{password}@{host}:{port}/{database}')

In [23]:
# Creating the table venues with constraints
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {schema}.venues (
    venue_id VARCHAR(10) NOT NULL UNIQUE,   -- PK: must be unique and not null
    district_id VARCHAR(20) NOT NULL,        -- FK: must be not null
    name VARCHAR(200) NOT NULL,             -- venue name: required
    district VARCHAR(200) NOT NULL,         -- district name: required

    category VARCHAR(200),
    cuisine VARCHAR(200),
    phone VARCHAR(50),
    address VARCHAR(200),
    latitude DECIMAL(9,6),
    longitude DECIMAL(9,6),
    website VARCHAR(200),
    opening_hours_dict JSONB,
    opening_hours VARCHAR(500),
    postal_code VARCHAR(10),
    neighborhood VARCHAR(200),
    takeaway VARCHAR(10),
    wheelchair_accessible VARCHAR(10),

    CONSTRAINT venues_pk PRIMARY KEY (venue_id),
    CONSTRAINT venues_district_fk FOREIGN KEY (district_id)
        REFERENCES {schema}.districts(district_id)
        ON DELETE RESTRICT
        ON UPDATE CASCADE
);
"""

# Execute the query to create empty table
with engine.connect() as conn:   
    conn.execute(text(create_table_query))
    conn.commit()

In [24]:
# Read the CSV file
df = pd.read_csv(
    "/Users/giovanigoltara/Documents/webeet/layered-populate-data-pool-da/venues/sources/berlin_venues_cleaned.csv",
    sep=None,          # infer delimiter
    engine="python"    # needed when sep=None
)

#  Send the DataFrame to the database using .to_sql()
df.to_sql(
    f'venues',        
    engine,
    schema=schema,
    if_exists='append', 
    index=False
)

print("DataFrame sent to PostgreSQL using .to_sql() with psycopg2!")

DataFrame sent to PostgreSQL using .to_sql() with psycopg2!
